# EXTRACCIÓN TRANSFORMACIÓN Y CARGA (ETL) #
El objetivo es realizar el proceso de extracción, transformación y carga (ETL) de los datos de siniestros viales en la Ciudad de Buenos Aires, para poder disponer de ellos en un formato adecuado para su posterior análisis y visualización. Este proceso permitirá acceder a la información de los siniestros y las víctimas de forma estructurada y limpia, así como resolver los posibles problemas de calidad y consistencia de los datos.

## 1. EXTRACCIÓN

Importación de Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Lectura de los datasets 

In [6]:
df_homicidios = pd.read_excel("..\data\homicidios.xlsx",sheet_name = "HECHOS",na_values=['SD', 'sd'])
df_vic_hom    = pd.read_excel("..\data\homicidios.xlsx",sheet_name = "VICTIMAS",na_values=['SD', 'sd'])
df_lesiones   = pd.read_excel("..\data\lesiones.xlsx", sheet_name = "HECHOS",na_values=['SD', 'sd'])
df_vic_les    = pd.read_excel("..\data\lesiones.xlsx" , sheet_name = "VICTIMAS",na_values=['SD', 'sd'])


## 2. TRANSFORMACIÓN

### 2.1. Homicidios_Hechos

In [8]:
# Se muestran 3 filas de la Tabla
display(df_homicidios.sample(3))

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
9,2016-0013,1,2016-02-10,2016,2,10,11:30:00,11.0,AV ENTRE RIOS 1366,AVENIDA,...,1366.0,NaN,ENTRE RIOS AV. 1366,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,PEATON-AUTO,PEATON,AUTO
650,2021-0052,1,2021-06-20,2021,6,20,05:40:00,5.0,"PAZ, GRAL. AV. 14723",GRAL PAZ,...,14723.0,NaN,"PAZ, GRAL. AV. 14723",8,Point (97067.24775528 94359.84584291),-58.49530249,-34.68010798,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
475,2019-0059,1,2019-07-13,2019,7,13,22:35:00,22.0,15 DE NOVIEMBRE DE 1889 Y SANTIAGO DEL ESTERO,CALLE,...,NaN,SANTIAGO DEL ESTERO,15 DE NOVIEMBRE DE 1889 y SANTIAGO DEL ESTERO,1,Point (107261.50953067 99806.16899068),-58.38410880,-34.63099197,MOTO-PASAJEROS,MOTO,PASAJEROS


In [9]:
# Se remueve las columnas que no se usarán
df_homicidios.drop(columns=['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO', 'Calle', 
                            'Altura', 'Cruce', 'Dirección Normalizada', 'PARTICIPANTES','VICTIMA'], inplace=True)

In [10]:
#Informe de Homicidios_Hechos
print(df_homicidios.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   HH             695 non-null    float64       
 4   TIPO_DE_CALLE  696 non-null    object        
 5   COMUNA         696 non-null    int64         
 6   XY (CABA)      696 non-null    object        
 7   pos x          696 non-null    object        
 8   pos y          696 non-null    object        
 9   ACUSADO        673 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 54.5+ KB
None


In [11]:
# Cantidad de valores nulos
df_homicidios.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
XY (CABA)         0
pos x             0
pos y             0
ACUSADO          23
dtype: int64

In [12]:
# remplazar la comuna 0 a nulos
df_homicidios.loc[df_homicidios['COMUNA'] == '0' ,'COMUNA'] = np.nan


# remplazar los "." por nulos en latitud y longitud
df_homicidios.loc[df_homicidios['pos x'] == '.' ,'pos x'] = np.nan
df_homicidios.loc[df_homicidios['pos y'] == '.' ,'pos y'] = np.nan
df_homicidios.loc[df_homicidios['XY (CABA)'] == 'Point (. .)' ,'XY (CABA)'] = np.nan

In [13]:
df_homicidios.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
XY (CABA)        14
pos x            12
pos y            12
ACUSADO          23
dtype: int64

In [14]:
# cambio a formato categórico
df_homicidios['HH'] = df_homicidios['HH'].astype('category')
df_homicidios['TIPO_DE_CALLE'] = df_homicidios['TIPO_DE_CALLE'].astype('category')
df_homicidios['COMUNA'] = df_homicidios['COMUNA'].astype('category')
df_homicidios['ACUSADO'] = df_homicidios['ACUSADO'].astype('category')

# cambio a float los valores de latitud y longitud
df_homicidios['pos x'] = df_homicidios['pos x'].astype('float')
df_homicidios['pos y'] = df_homicidios['pos y'].astype('float')

# Se verifica el tipo de dato
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   HH             695 non-null    category      
 4   TIPO_DE_CALLE  696 non-null    category      
 5   COMUNA         696 non-null    category      
 6   XY (CABA)      682 non-null    object        
 7   pos x          684 non-null    float64       
 8   pos y          684 non-null    float64       
 9   ACUSADO        673 non-null    category      
dtypes: category(4), datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 37.4+ KB


In [15]:
# Se establece nuevos nombres para los campos
nuevos_nombres = {'ID': 'ID_HECHO','pos x':'LONGITUD','pos y':'LATITUD',
                  'HH':'HORA','ACUSADO':'VEHICULO_ACUSADO',
                  'Rango_etario ':'RANGO_ETARIO'}
                 
# Remplazar nombres
df_homicidios = df_homicidios.rename(columns=nuevos_nombres)

In [57]:
# comprobando si hay registros duplicados
df_homicidios.duplicated().sum()

0

In [26]:
df_homicidios.sample(5)

,ID_HECHO,N_VICTIMAS,FECHA,HORA,TIPO_DE_CALLE,COMUNA,XY (CABA),LONGITUD,LATITUD,VEHICULO_ACUSADO
376,2018-0102,1,2018-09-21,0.0,CALLE,4.0,Point (104810.02928662 98097.69340075),-58.410834,-34.646407,AUTO
641,2021-0043,1,2021-05-22,18.0,AVENIDA,9.0,Point (95975.76718859 97082.39835843),-58.507200,-34.655562,AUTO
679,2021-0081,1,2021-11-02,13.0,AUTOPISTA,4.0,Point (107979.30419282 99027.13636949),-58.376274,-34.638009,CARGAS
646,2021-0048,1,2021-06-03,9.0,AVENIDA,12.0,Point (97525.48112402 106595.29222041),-58.490266,-34.569814,AUTO
348,2018-0074,1,2018-07-14,6.0,AVENIDA,7.0,Point (99438.84176266 99501.28380301),-58.469420,-34.633765,CARGAS


In [27]:
# Guardamos el DF listo para su análisis
df_homicidios.to_csv('../data/df_homicidios.csv',index=False)

### 2.2. Homicidios_Victimas

In [28]:
# Se muestran 3 registros de la Tabla
display(df_vic_hom.sample(3))

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
715,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27.0,2022-01-02 00:00:00
192,2017-0055,2017-05-09,2017,5,9,CONDUCTOR,AUTO,MASCULINO,32.0,2017-05-09 00:00:00
462,2019-0027,2019-03-28,2019,3,28,CICLISTA,BICICLETA,MASCULINO,37.0,2019-03-28 00:00:00


In [29]:
#Informe de Homicidios_Víctimas
print(df_vic_hom.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  706 non-null    object        
 6   VICTIMA              708 non-null    object        
 7   SEXO                 711 non-null    object        
 8   EDAD                 664 non-null    float64       
 9   FECHA_FALLECIMIENTO  649 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 56.1+ KB
None


In [30]:
# Se remueve las columnas que no se usarán
df_vic_hom.drop(columns=['AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO'], inplace=True)

df_vic_hom['GRAVEDAD'] = 'FATAL'

In [31]:
# función que regresa 1 si la edad es menor a 18, 2 si la edad esta entre 18 y 29
# 3 si la edad esta entre 30 y 39, 4 si la edad esta entre 40 a 49 , 5 si la edad es de 50 a 59
# y 6 si la edad es igual o mayor a 60 
def rango_etario(edad):
    if edad < 18:
        return "1_ 0 a 17 años"
    elif 18 <= edad <= 29:
        return "2_ 18 a 29 años"
    elif 30 <= edad <= 39:
        return "3_ 30 a 39 años"
    elif 40 <= edad <= 49:
        return "4_ 45 a 49 años"
    elif 50 <= edad <= 59:
        return "5_ 50 a 59 años"
    elif edad >= 60:
        return "6_ 60 años o mas"
    else:
        return np.nan

In [32]:
# Rango Etario
df_vic_hom.loc[df_vic_hom['EDAD'] == 'SD', 'EDAD'] = np.nan
df_vic_hom.loc[df_vic_hom['EDAD'] == 'sd', 'EDAD'] = np.nan
df_vic_hom['EDAD'] = df_vic_hom['EDAD'].astype('float')
df_vic_hom['RANGO_ETARIO'] = df_vic_hom['EDAD'].apply(lambda x: rango_etario(x))
df_vic_hom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID_hecho      717 non-null    object        
 1   FECHA         717 non-null    datetime64[ns]
 2   ROL           706 non-null    object        
 3   VICTIMA       708 non-null    object        
 4   SEXO          711 non-null    object        
 5   EDAD          664 non-null    float64       
 6   GRAVEDAD      717 non-null    object        
 7   RANGO_ETARIO  664 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 44.9+ KB


In [33]:
# cambio a formato categórico
df_vic_hom['ID_hecho'] = df_vic_hom['ID_hecho'].astype('category')
df_vic_hom['ROL'] = df_vic_hom['ROL'].astype('category')
df_vic_hom['VICTIMA'] = df_vic_hom['VICTIMA'].astype('category')
df_vic_hom['SEXO'] = df_vic_hom['SEXO'].astype('category')
df_vic_hom['RANGO_ETARIO'] = df_vic_hom['RANGO_ETARIO'].astype('category')


df_vic_hom.drop(columns='EDAD',inplace=True)

df_vic_hom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID_hecho      717 non-null    category      
 1   FECHA         717 non-null    datetime64[ns]
 2   ROL           706 non-null    category      
 3   VICTIMA       708 non-null    category      
 4   SEXO          711 non-null    category      
 5   GRAVEDAD      717 non-null    object        
 6   RANGO_ETARIO  664 non-null    category      
dtypes: category(5), datetime64[ns](1), object(1)
memory usage: 38.0+ KB


In [34]:
# CAMBIAR EL NOMBRE A LAS COLUMNAS 
nuevos_nombres = {'ID_hecho':'ID_HECHO','ROL':'ROL_VICTIMA','VICTIMA':'VEHICULO_VICTIMA','SEXO':'SEXO_VICTIMA'}

# Remplazar nombres
df_vic_hom = df_vic_hom.rename(columns=nuevos_nombres)
df_vic_hom.columns

Index(['ID_HECHO', 'FECHA', 'ROL_VICTIMA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA',
       'GRAVEDAD', 'RANGO_ETARIO'],
      dtype='object')

In [35]:
#cambia el ROL_VICTIMA  =  PASAJERO_ACOMPAÑANTE por PASAJERO
df_vic_hom['ROL_VICTIMA'] = df_vic_hom['ROL_VICTIMA'].replace('PASAJERO_ACOMPAÑANTE','PASAJERO')

In [36]:
# Guardamos el DF listo para su análisis
df_vic_hom.to_csv('../data/df_vic_hom.csv',index=False)

### 2.3. Lesiones_Hechos

In [37]:
# Se muestran 3 registros de la Tabla
display(df_lesiones.sample(3))

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
13938,LC-2020-0506628,1,2020,9,4,2020-09-04 00:00:00,02:35:00 p.m.,14.0,"ANTARTIDA ARGENTINA AV. y PY, COMODORO AV.",1,...,-34.590707,MOTO,AUTO,MOTO-AUTO,x,x,0,0,0,NaN
17435,LC-2021-0045520,1,2021,1,31,2021-01-31 00:00:00,21:30:00,21.0,"PERON, EVA AV. y CURAPALIGUE",7,...,-34.634917,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
13997,LC-2020-0513611,1,2020,9,10,2020-09-10 00:00:00,12:50:00 a.m.,0.0,SAENZ AV. 1440,4,...,-34.658088,NaN,NaN,SD-SD,0,0,0,0,0,NaN


In [39]:
#se borra las columnas que no se usará
df_lesiones.drop(columns=['aaaa', 'mm', 'dd','hora','direccion_normalizada',
                          'calle','altura','cruce','participantes','moto',
                          'auto','transporte_publico','camion','otra_direccion',
                          'ciclista','victima','gravedad'], inplace=True)

In [40]:
# Pasar el nombre de las columnas a mayusculas
df_lesiones.columns = df_lesiones.columns.str.upper()

# CAMBIAR NOMBRE DE LA COLUMNA FRANJA HORARIA
df_lesiones = df_lesiones.rename(columns={'ID':'ID_HECHO'})
df_lesiones = df_lesiones.rename(columns={'FRANJA_HORA':'HORA'})
df_lesiones = df_lesiones.rename(columns={'TIPO_CALLE':'TIPO_DE_CALLE'})
df_lesiones = df_lesiones.rename(columns={'GEOCODIFICACION_CABA':'XY (CABA)'})
df_lesiones = df_lesiones.rename(columns={'ACUSADO':'VEHICULO_ACUSADO'})
df_lesiones = df_lesiones.rename(columns={'LATUTID':'LATITUD'})

df_lesiones.sample(3)	

,ID_HECHO,N_VICTIMAS,FECHA,HORA,COMUNA,TIPO_DE_CALLE,XY (CABA),LONGITUD,LATITUD,VEHICULO_ACUSADO
22209,LC-2021-0523423,1,2021-10-21 00:00:00,19.0,5,CALLE,Point (104654.479815148 100956.080341333),-58.412546,-34.620641,NaN
964,LC-2019-0085001,1,2019-02-09 00:00:00,7.0,3,NaN,Point (106309.44641145359491929 101214.7196850...,-58.394502,-34.618301,NaN
14539,LC-2020-0566235,1,2020-10-15 00:00:00,1.0,1,AVENIDA,Point (107729.64980728 101807.899528158),-58.379022,-34.612944,NaN


In [41]:
df_lesiones['FECHA'].replace("'","", inplace=True)
df_lesiones['FECHA'] = df_lesiones['FECHA'].astype('datetime64[ns]')

# Remplazar "No especificada" por nulos
df_lesiones.loc[df_lesiones['COMUNA'] == 'No Especificada', 'COMUNA'] = np.nan

In [58]:
#Se verifica que FECHA ya no incluye la hora
display(df_lesiones.sample(3))

,ID_HECHO,N_VICTIMAS,FECHA,HORA,COMUNA,TIPO_DE_CALLE,XY (CABA),LONGITUD,LATITUD,VEHICULO_ACUSADO
7892,LC-2019-0609055,1,2019-09-23,18.0,13,NaN,Point (99724.15893275341659319 110270.27910080...,-58.466304,-34.536689,NaN
17272,LC-2021-0029570,1,2021-01-20,17.0,1,CALLE,Point (107417.64685477 103208.154790465),-58.382436,-34.600324,AUTO
20646,LC-2021-0402076,1,2021-08-12,19.0,10,CALLE,Point (96970.9243619105 100644.603830319),-58.496331,-34.623455,NaN


In [42]:
df_lesiones.columns

Index(['ID_HECHO', 'N_VICTIMAS', 'FECHA', 'HORA', 'COMUNA', 'TIPO_DE_CALLE',
       'XY (CABA)', 'LONGITUD', 'LATITUD', 'VEHICULO_ACUSADO'],
      dtype='object')

In [43]:
# Guardamos el DF listo para su análisis
df_lesiones.to_csv('../data/df_lesiones.csv',index=False)

### 2.4. Lesiones_Victimas

In [44]:
#Se meustra 3 registros 
display(df_vic_les.sample(3))

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
7148,LC-2019-0167392,2019,3,18,2019-03-18,NaN,Mujer,3.0,NaN
1321,LC-2019-0099968,2019,2,15,2019-02-15,NaN,Mujer,46.0,NaN
18049,LC-2020-0608466,2020,11,11,2020-11-11,MOTO,Mujer,19.0,NaN


In [45]:
# Pasar los nombres de las columnas a mayúsculas
df_vic_les.columns = df_vic_les.columns.str.upper()

# Cambiar nombres a las columnas para coincidir con el df_homicidios
df_vic_les = df_vic_les.rename(columns={'ID HECHO':'ID_HECHO','FECHA ':'FECHA','SEXO':'SEXO_VICTIMA'})

df_vic_les.columns

Index(['ID_HECHO', 'AAA', 'MM', 'DD', 'FECHA', 'VEHICULO_VICTIMA',
       'SEXO_VICTIMA', 'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')

In [46]:
#Se borra las columnas que no se usará
df_vic_les.drop(columns=['AAA', 'MM', 'DD'], inplace=True)

# Columnas
df_vic_les.columns

Index(['ID_HECHO', 'FECHA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA', 'EDAD_VICTIMA',
       'GRAVEDAD'],
      dtype='object')

In [47]:
# Creamos la columna ROL_VICTIMA con nulls
df_vic_les['ROL_VICTIMA'] = np.nan

# remplazar los nulos de la columna GRAVEDAD por "LEVEMENTE HERIDO" -> Toda persona que requiere una atención médica mínima o nula
df_vic_les.loc[df_vic_les['GRAVEDAD'].isnull(), 'GRAVEDAD'] = 'LEVEMENTE HERIDO'

df_vic_les.head(3)

,ID_HECHO,FECHA,VEHICULO_VICTIMA,SEXO_VICTIMA,EDAD_VICTIMA,GRAVEDAD,ROL_VICTIMA
0,LC-2019-0000053,2019-01-01,NaN,Varon,57.0,LEVEMENTE HERIDO,NaN
1,LC-2019-0000063,2019-01-01,NaN,NaN,NaN,LEVEMENTE HERIDO,NaN
2,LC-2019-0000079,2019-01-01,NaN,Varon,NaN,LEVEMENTE HERIDO,NaN


***Nota:*** *SD = Sin datos sobre la gravedad de las lesiones provocadas. A efectos analíticos, los casos sin datos se corresponden con una alta probabilidad a casos leves.*

In [48]:
df_vic_les.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID_HECHO          27605 non-null  object        
 1   FECHA             27605 non-null  datetime64[ns]
 2   VEHICULO_VICTIMA  11333 non-null  object        
 3   SEXO_VICTIMA      25377 non-null  object        
 4   EDAD_VICTIMA      23134 non-null  float64       
 5   GRAVEDAD          27605 non-null  object        
 6   ROL_VICTIMA       0 non-null      float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 1.5+ MB


In [49]:
# Rango Etario
df_vic_les['EDAD_VICTIMA'] = df_vic_les['EDAD_VICTIMA'].astype('float')
df_vic_les['RANGO_ETARIO'] = df_vic_les['EDAD_VICTIMA'].apply(lambda x: rango_etario(x))

In [50]:
# Unificar en FEMENINO y MASCULINO
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'MUJER', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Mujer', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'mujer ', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'mujer', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Mujer ', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Varon', 'SEXO_VICTIMA'] = 'MASCULINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'varon', 'SEXO_VICTIMA'] = 'MASCULINO'

In [51]:
# imprime valores unicos de la columna SEXO_VICTIMA
df_vic_les['SEXO_VICTIMA'].unique()

array(['MASCULINO', nan, 'FEMENINO'], dtype=object)

In [53]:
# Guardamos el DF listo para su análisis
df_vic_les.to_csv('../data/df_vic_les.csv',index=False)